Note : We Referred source code at  https://github.com/GauriNaik826/PUMA-PLASMA-ACL

In [1]:
!pip install evaluate
!pip install bert_score
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

In [40]:
import evaluate
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import json, os, sys, math
from torch.optim import AdamW
from scipy.spatial.distance import cosine
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from transformers import (
    # BartTokenizer, BartForConditionalGeneration,
    # GPT2LMHeadModel, AutoModelForSeq2SeqLM, AutoTokenizer,
    BertTokenizer, BertModel,
    # Seq2SeqTrainer, Seq2SeqTrainingArguments,
    # DataCollatorForSeq2Seq,
    # get_linear_schedule_with_warmup,
    RobertaForSequenceClassification, RobertaTokenizer
)
from bert_score import score as bert_score
from rouge_score import rouge_scorer

In [2]:
!mkdir classifier

In [3]:
import gdown
import zipfile


#download the dataset
url = 'https://drive.google.com/uc?id=1OziFad2e4YYdECqptnBLvSsLwuNp8LoN'
output = 'PUMA.zip'
gdown.download(url, output, quiet=False)

with zipfile.ZipFile('PUMA.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

Downloading...
From: https://drive.google.com/uc?id=1OziFad2e4YYdECqptnBLvSsLwuNp8LoN
To: /content/PUMA.zip
100%|██████████| 3.26M/3.26M [00:00<00:00, 232MB/s]


# Classifier

## Dataset

In [ ]:
# class Classifier_CustomDataset(Dataset):
#     def __init__(self, data, tokenizer, max_length=512):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#         self.label_map = {"EXPERIENCE": 0, "SUGGESTION": 1, "INFORMATION": 2, "CAUSE": 3, "QUESTION": 4}

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         input_text = self.data[idx]['Summary']
#         label = self.data[idx]['Perspective']

#         # Encode label to integer
#         label_encoded = self.label_map[label]


#         inputs = self.tokenizer(
#             input_text,
#             padding="max_length",
#             max_length=self.max_length,
#             truncation=True,
#             return_tensors="pt"
#         )

#         # Return input_ids, attention_mask, and label
#         return {
#             "input_ids": inputs["input_ids"].squeeze(),
#             "attention_mask": inputs["attention_mask"].squeeze(),
#             "label": torch.tensor(label_encoded)  # Return integer label directly
#         }

## Helper functions

In [ ]:

# def classifier_create_dataloader(train_dataset,valid_dataset, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE ):
#     train_dataloader= DataLoader(dataset = train_dataset, batch_size = TRAIN_BATCH_SIZE, shuffle= True)
#     valid_dataloader = DataLoader(dataset = valid_dataset, batch_size = VALID_BATCH_SIZE, shuffle= True)
#     return train_dataloader , valid_dataloader

# def classifier_test_create_dataloader(test_dataset, TEST_BATCH_SIZE ):
#     test_dataloader= DataLoader(dataset = test_dataset, batch_size = TEST_BATCH_SIZE, shuffle= True)
#     return test_dataloader

# def classifier_validation(valid_dataloader, model, VALID_BATCH_SIZE, optimizer, scheduler,device):
#     print("Validation processing...")
#     model.eval()
#     valid_losses = []

#     with torch.no_grad():
#         for i,batch in enumerate(tqdm(valid_dataloader)):

#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['label'].to('cuda')
#             with autocast():
#               output = model(input_ids= input_ids,attention_mask=attention_mask,labels=labels)
#               loss = output.loss


#             #print(f"_________________ValidBatch: {i}/{len(valid_dataloader)} || ValidLoss: {loss}_____________________")
#             valid_losses.append(loss.item())

#     valid_loss = np.mean(valid_losses) if len(valid_losses) > 0 else 0.0  # Calculate mean of valid_losses
#     print(f"Validation loss: {valid_loss}")
#     return valid_loss

# def classifier_train(train_dataloader,eval_dataloader,optimizer,scheduler,device,start_epoch,last_epoch,EPOCHS,best_loss):
#   scaler = GradScaler()
#   for epoch in range(start_epoch,start_epoch+ EPOCHS):
#       model.train()
#       print(f"#"*50 + f"Epoch: {epoch}" + "#"*50)
#       train_losses = []
#       for i,batch in enumerate(tqdm(train_dataloader)):

#           input_ids = batch['input_ids'].to(device)
#           attention_mask = batch['attention_mask'].to(device)
#           labels = batch['label'].to(device)

#           optimizer.zero_grad()

#           with autocast():
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss

#           scaler.scale(loss).backward()
#           scaler.step(optimizer)
#           scaler.update()
#           scheduler.step()
#           train_losses.append(loss.detach())

#       train_losses = [loss.item() for loss in train_losses]
#       train_loss = np.mean(train_losses)
#       print(f"Train loss: {train_loss} for epoch : {epoch}")
#       print(f"LR : {scheduler.get_last_lr()[0]:.8f}")
#       #Save the checkpoint for the batch with best loss after comparing with validation
#       valid_loss = classifier_validation(eval_dataloader, model, VALID_BATCH_SIZE, optimizer, scheduler,device)

#       if valid_loss < best_loss:
#               best_loss = valid_loss
#               state_dict = {
#                   'model_state_dict': model.state_dict(),
#                   'optim_state_dict': optimizer.state_dict(),
#                   'sched_state_dict': scheduler.state_dict(),
#                   'loss': best_loss,
#                   'epoch': last_epoch
#               }

#               torch.save(state_dict, f"/kaggle/working/classifier/best_ckpt_epoch={epoch}_valid_loss={round(best_loss, 4)}.ckpt")
#               print("*"*10 + "Current best checkpoint is saved." + "*"*10)



## Training

Load

In [ ]:
# classifier_train_path = "/kaggle/working/dataset/PUMA/train.json"
# classifier_valid_path = "/kaggle/working/dataset/PUMA/valid.json"

In [ ]:
# with open(classifier_train_path, 'r') as json_file:
#     train_data = json.load(json_file)

# with open(classifier_valid_path, 'r') as json_file:
#     valid_data = json.load(json_file)

Preprocess

In [ ]:
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
# new_train_data = []
# new_val_data = []

# for i in train_data:
#   for j in i['labelled_summaries']:
#     new_train_data.append({'Perspective': j.split('_')[0],'Summary' : i['labelled_summaries'][j]})
# print(new_train_data[0])
# for i in valid_data:
#   for j in i['labelled_summaries']:
#     new_val_data.append({'Perspective': j.split('_')[0],'Summary' : i['labelled_summaries'][j]})
# print(new_val_data[0])

# train_dataset = Classifier_CustomDataset(new_train_data,tokenizer)
# eval_dataset = Classifier_CustomDataset(new_val_data,tokenizer)

In [ ]:
# perspective_count = {"EXPERIENCE": 0, "SUGGESTION": 0, "INFORMATION": 0, "CAUSE": 0, "QUESTION": 0}
# for i in new_train_data:
#   perspective_count[i['Perspective']] += 1
# print('training_data class balance',perspective_count)
# perspective_count = {"EXPERIENCE": 0, "SUGGESTION": 0, "INFORMATION": 0, "CAUSE": 0, "QUESTION": 0}
# for i in new_val_data:
#   perspective_count[i['Perspective']] += 1
# print('validation_data class balance',perspective_count)


In [ ]:
# VALID_BATCH_SIZE = 40
# TRAIN_BATCH_SIZE = 40
# LR =  1e-05
# EPOCHS = 15
# training_steps = (len(train_dataset)*EPOCHS)//TRAIN_BATCH_SIZE
# WARMUP_STEPS = training_steps//2
# # NUM_OPTIM_STEPS = args.num_optim_steps
# best_loss = sys.float_info.max
# print("best_loss",best_loss)
# last_epoch = 0

In [ ]:
# print(WARMUP_STEPS)

In [ ]:
# model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)  # Change num_labels to match your number of classes
# train_dataloader, eval_dataloader = classifier_create_dataloader(train_dataset, eval_dataset, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE)


In [ ]:
# optimizer = AdamW(model.parameters(), lr=LR)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=len(train_dataloader) * EPOCHS)

# start_epoch = last_epoch+1
# num_batches = len(train_dataloader)


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
# model.to(device)

In [ ]:
# del model
# del optimizer
# del scheduler
# del train_dataloader
# del eval_dataloader

# # import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# classifier_train(train_dataloader,eval_dataloader,optimizer,scheduler,device,start_epoch,last_epoch,EPOCHS,best_loss)

In [ ]:
# !zip -r /kaggle/working/classifier.zip /kaggle/working/classifier

# PLASMA

## Dataset

In [24]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer,max_length=1024):

        self.data = []
        self.tokenizer = tokenizer
        self.max_length = max_length
        for i in data:
            new_rec = {}
            for j in i['labelled_summaries']:
                # print(j.split('_')[0])
                new_rec['Perspective'] = j.split('_')[0]
                new_rec['Summary'] = i['labelled_summaries'][j]
                new_rec['answers'] = i['answers']
                new_rec['question'] = i['question']
                self.data.append(new_rec)
                # for k in new_rec:
                    # print(k, new_rec[k])
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        answers = self.data[idx]['answers']
        # target_text = self.data[idx]['Summary']
        defn = ""
        start_with = ""
        tone_attribute = ""
        l = []
        target_text = ''
        if self.data[idx]['Perspective'].strip() ==  "SUGGESTION":
            defn = "Defined as advice or recommendations to assist users in making informed medical decisions, solving problems, or improving health issues."
            start_with =  "It is suggested"
            tone_attribute = "Advisory, Recommending"
            l =  ["Advisory", "Recommending", "Cautioning", "Prescriptive", "Guiding","Prescriptive"]
            if len(set(self.data[idx]['Summary'].split(" ")[:5]).intersection(set(start_with.split()))) > 3  :
                target_text = self.data[idx]['Summary']
            else:
                target_text = start_with + " " +self.data[idx]['Summary']
        if self.data[idx]['Perspective'].strip() == "INFORMATION":
            defn = "Defined as knowledge about diseases, disorders, and health-related facts, providing insights into symptoms and diagnosis."
            start_with =  "For information purposes"
            tone_attribute = "Informative, Educational"
            l =  ["Clinical", "Scientific","Informative", "Educational","Factual", "Informing","Academic","Analytical"]
            if len(set(self.data[idx]['Summary'].split(" ")[:5]).intersection(set(start_with.split()))) >= 2:
                target_text = self.data[idx]['Summary']
            else:
                target_text = start_with + " " +self.data[idx]['Summary']
        if self.data[idx]['Perspective'].strip() == "EXPERIENCE":
            defn = "Defined as individual experiences, anecdotes, or firsthand insights related to health, medical treatments, medication usage, and coping strategies"
            start_with =  "In user's experience"
            tone_attribute = "Personal, Narrative"
            l =  ["Personal", "Narrative", "Introspective", "Exemplary", "Insightful", "Emotional"]
            if len(set(self.data[idx]['Summary'].split(" ")[:5]).intersection(set(start_with.split()))) >= 2:
                target_text = self.data[idx]['Summary']
            else:
                target_text = start_with + " " +self.data[idx]['Summary']
        if self.data[idx]['Perspective'].strip() == "CAUSE":
            defn = "Defined as reasons responsible for the occurrence of a particular medical condition, symptom, or disease"
            start_with =  "Some of the causes"
            tone_attribute = "Explanatory, Causal"
            l =  ["Diagnostic", "Explanatory", "Causal","Due to", "Resulting from", "Attributable to" ]
            if len(set(self.data[idx]['Summary'].split(" ")[:5]).intersection(set(start_with.split()))) >= 2:
                target_text = self.data[idx]['Summary']
            else:
                target_text = start_with + " " +self.data[idx]['Summary']
        if self.data[idx]['Perspective'].strip() == "QUESTION":
            defn = "Defined as inquiry made for deeper understanding."
            start_with =  "It is inquired"
            tone_attribute = "Seeking Understanding"
            l =  ["Inquiry", "Rhetorical", "Exploratory Questioning", "Clarifying Inquiry", "Problem-Solving Deliberation"]
            if len(set(self.data[idx]['Summary'].split(" ")[:5]).intersection(set(start_with.split()))) >= 2:
                target_text = self.data[idx]['Summary']
            else:
                target_text = start_with + " " +self.data[idx]['Summary']

        non_empty_sentences = ' '.join([sentence.replace('\n', '') for sentence in answers])

        task_prefix = "Adhering to the condition of 'begin summary with' and 'tone of summary' and summarize according to "+self.data[idx]['Perspective'].strip()+ " and start the summary with '"+ start_with.strip()+ "'. Maintain summary tone as " + tone_attribute.strip()+ ". Definition of perspective: "+ defn.strip().lower() + " Content to summarize: "+ non_empty_sentences +" Question: "+ self.data[idx]['question'].strip()+"."

        inputs = self.tokenizer(task_prefix, padding="max_length", max_length=self.max_length, truncation=True, return_tensors="pt")
        labels = self.tokenizer(target_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        labels_input_ids = labels["input_ids"].squeeze()
        labels_input_ids[labels_input_ids == self.tokenizer.pad_token_id] = -100
        # return {'input': task_prefix,'label': target_text}
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels_input_ids,
            "perspective": self.data[idx]['Perspective'],
            "Summary": self.data[idx]['Summary']
        }

def create_dataloader(train_dataset,valid_dataset, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE ):
    train_dataloader= DataLoader(dataset = train_dataset, batch_size = TRAIN_BATCH_SIZE, shuffle= True)
    valid_dataloader = DataLoader(dataset = valid_dataset, batch_size = VALID_BATCH_SIZE, shuffle=True)

    return train_dataloader , valid_dataloader

def test_create_dataloader(test_dataset, TEST_BATCH_SIZE ):
    test_dataloader= DataLoader(dataset = test_dataset, batch_size = TEST_BATCH_SIZE, shuffle= False)
    return test_dataloader


## Helper functions

In [25]:
import gdown

url = 'https://drive.google.com/uc?id=1lxWVnj8aFwgcA8elR8lXmv-fajvvLWNT'
output = 'classifier_for_plasma.ckpt'
gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1lxWVnj8aFwgcA8elR8lXmv-fajvvLWNT
From (redirected): https://drive.google.com/uc?id=1lxWVnj8aFwgcA8elR8lXmv-fajvvLWNT&confirm=t&uuid=1d906ea9-a952-47cc-b518-517096e9d19d
To: /content/classifier_for_plasma.ckpt
100%|██████████| 1.50G/1.50G [00:19<00:00, 75.9MB/s]


'classifier_for_plasma.ckpt'

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [27]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [28]:
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5).to(device)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
ckpt_path = "/content/classifier_for_plasma.ckpt"
if os.path.exists(ckpt_path):
    print("Loading the trained checkpoint...")
    ckpt = torch.load(ckpt_path, weights_only=False)
    roberta_model.load_state_dict(ckpt['model_state_dict'])
    print("The inference will start with the specified checkpoint.")

Loading the trained checkpoint...
The inference will start with the specified checkpoint.


In [32]:
def get_bert_embedding(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()


def calculate_rouge_score_for_each_phrase(predictions, references):
    # rouge = Rouge()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_l_f1_scores = []

    for prediction, reference in zip(predictions, references):
        # scores = rouge.get_scores(prediction.lower(), reference.lower())[0]
        # rouge_l_f1 = scores["rouge-1"]["f"]
        scores = scorer.score(reference.lower(), prediction.lower())
        rouge_l_f1 = scores['rougeL'].fmeasure
        rouge_1_f1 = scores['rouge1'].fmeasure
        rouge_l_f1_scores.append(rouge_l_f1)

    return rouge_l_f1_scores

def score_all_phrases(summary, phrases):
    start_of_summary = ' '.join(summary.split()[:4])


    predictions = [start_of_summary] * len(phrases)
    references = phrases

    rouge_l_f1_results = calculate_rouge_score_for_each_phrase(predictions, references)

    phrase_scores = dict(zip(phrases, rouge_l_f1_results))

    return phrase_scores

def Ep(generated_summary):
    inputs = roberta_tokenizer(generated_summary, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = roberta_model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predictions = outputs.logits.argmax(dim=-1)

    class_labels = {0: "EXPERIENCE", 1: "SUGGESTION", 2: "INFORMATION", 3: "CAUSE", 4: "QUESTION"}
    predicted_label = class_labels[predictions[0].item()]

    final ={}
    for i in range(0,5):
        final[class_labels[i]] = probabilities[0][i].cpu().numpy().item()

    return final


def Es(generated_summary):
    phrases = [
        "In user's experience…",
        "It is suggested",
        "For information purposes",
        "Some of the causes",
        "It is inquired"
    ]

    phrase_scores = score_all_phrases(generated_summary, phrases)
    return phrase_scores

def Et(generated_summary):
    l_sugg =  ["Advisory", "Recommending", "Cautioning", "Prescriptive", "Guiding","Prescriptive"]
    l_exp = ["Personal", "Narrative", "Introspective", "Exemplary", "Insightful", "Emotional"]
    l_info =  ["Clinical", "Scientific","Informative", "Educational","Factual", "Informing","Academic","Analytical"]
    l_cause =  ["Diagnostic", "Explanatory", "Causal","Due to", "Resulting from", "Attributable to" ]
    l_qs =  ["Inquiry", "Rhetorical", "Exploratory Questioning", "Clarifying Inquiry", "Problem-Solving Deliberation"]


    summary_embedding = get_bert_embedding(generated_summary)

    cosine_similarities = {}

    for label, word_list in zip(['sugg', 'exp', 'info', 'cause', 'qs'], [l_sugg, l_exp, l_info, l_cause, l_qs]):
        combined_text = ' '.join(word_list)
        word_embedding = get_bert_embedding(combined_text)
        similarity = 1 - cosine(summary_embedding.cpu().detach().numpy(), word_embedding.cpu().detach().numpy())
        cosine_similarities[label] = similarity

    return cosine_similarities

def compute_custom_loss(model, input_text, input_attention, perspective):
    model.eval()
    outputs = model.generate(input_ids=input_text,attention_mask=input_attention,num_beams=5, max_new_tokens=100,temperature=0.9)
    generated_summary = tokenizer.decode(outputs[0])
    if len(generated_summary) <= 0:
        generated_summary = 'None'

    Ep_dict = Ep(generated_summary)
    Es_dict = Es(generated_summary)
    Et_dict = Et(generated_summary)


    alpha = 0.7
    beta = 0.3
    gamma = 0.5

    E_X = {
        "EXPERIENCE": alpha * Ep_dict["EXPERIENCE"] + beta * Es_dict["In user's experience…"] + gamma * Et_dict['exp'],
        "SUGGESTION": alpha * Ep_dict["SUGGESTION"] + beta * Es_dict["It is suggested"] + gamma * Et_dict['sugg'],
        "INFORMATION": alpha * Ep_dict["INFORMATION"] + beta * Es_dict["For information purposes"] + gamma * Et_dict['info'],
        "CAUSE": alpha * Ep_dict["CAUSE"] + beta * Es_dict["Some of the causes"] + gamma * Et_dict['cause'],
        "QUESTION": alpha * Ep_dict["QUESTION"] + beta * Es_dict["It is inquired"] + gamma * Et_dict['qs']
    }

    # Compute the exponential of E(X) for normalization
    exp_E_X = {k: math.exp(-1/(v)) for k, v in E_X.items()}

    # Compute the sum of the exponentials for normalization
    Z = sum(exp_E_X.values())

    # Calculate P(X) for each perspective
    P_X = {k: v / Z for k, v in exp_E_X.items()}


    Y = {"EXPERIENCE": 0, "SUGGESTION": 0, "INFORMATION": 0, "CAUSE": 0, "QUESTION": 0}
    Y[perspective[0]] = 1

    P_X_tensor = torch.tensor([P_X[k] for k in P_X])
    Y_tensor = torch.tensor([Y[k] for k in Y])

    loss = -torch.sum(Y_tensor * torch.log(P_X_tensor))
    generated_summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return loss,generated_summary


def validation(valid_dataloader, model, VALID_BATCH_SIZE, optimizer, scheduler,ret_pred = False):
    print("Validation processing...")
    get_sum = ret_pred
    model.eval()
    valid_losses = []
    gen = []
    actual =[]
    t = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            output = model(input_ids= input_ids,attention_mask=attention_mask,labels=labels)
            custom_loss,gen_sum = compute_custom_loss(model,input_ids,attention_mask, batch["perspective"])
            loss = output.loss + custom_loss

            gen.extend(gen_sum)
            actual.extend(batch['Summary'])
            valid_losses.append(loss.item())
            # print(gen)
            # print(actual)
            # print(outputs.loss,loss.item(),custom_loss)
            # break

    valid_loss = np.mean(valid_losses) if len(valid_losses) > 0 else 0.0
    print(f"Validation loss: {valid_loss}")
    if get_sum: return valid_loss,gen,actual
    else: return valid_loss

## Training

In [ ]:
!mkdir plasma

In [ ]:
train_file = "/kaggle/working/dataset/PUMA/train.json"
valid_file = "/kaggle/working/dataset/PUMA/valid.json"

with open(train_file, 'r') as f:
    train_data = json.load(f)

with open(valid_file, 'r') as f:
    valid_data = json.load(f)

In [ ]:
batch_size_train = 4
batch_size_valid = 4
# model_file = "t5-base"
learning_rate = 1e-5
warmup_steps = 16000
num_epochs = 3
ckpt_dir = "/kaggle/working/plasma"
ckpt_name = "model_ckpt"
model_file = "google/flan-t5-base"

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_file)
tokenizer = AutoTokenizer.from_pretrained(model_file)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
print(tokenizer.model_max_length)

512


In [ ]:
train_dataset = CustomDataset(train_data, tokenizer,max_length = tokenizer.model_max_length)
eval_dataset = CustomDataset(valid_data, tokenizer,max_length = tokenizer.model_max_length)

train_dataloader, eval_dataloader = create_dataloader(
    train_dataset, eval_dataset,
    batch_size_valid, batch_size_train
)

In [ ]:
list_loss_train = []
list_loss_valid = []
best_loss = sys.float_info.max
last_epoch = 0

In [ ]:
peft_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=8,
    token_dim=model.config.hidden_size
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()
model = peft_model

trainable params: 147,456 || all params: 247,725,312 || trainable%: 0.0595


In [ ]:
print(device)
model.to(device)
# if torch.cuda.device_count()>1:
#     print("using multiple gpus")
#     model = torch.nn.DataParallel(model)

cuda


PeftModelForSeq2SeqLM(
  (base_model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=len(train_dataloader) * num_epochs
)

In [ ]:
# model.to(device)
start_epoch = last_epoch + 1
# scaler = GradScaler()
for epoch in range(start_epoch, start_epoch + num_epochs):
    model.train()
    print(f"\n{'#' * 20} Epoch: {epoch} {'#' * 20}")
    train_losses = []

    for batch in tqdm(train_dataloader):
        # print("Model hidden size:", model.config.hidden_size)
        # print("Number of attention heads:", model.config.num_attention_heads)
        # print("Batch size in error case:", batch['input_ids'].size())
        # break

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # print("label typ : ",labels.dtype)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        custom_loss,gen_sum = compute_custom_loss(model,input_ids,attention_mask, batch["perspective"])

        # print(outputs.loss,custom_loss)
        # print(type(outputs.loss),type(custom_loss))
        loss = outputs.loss + custom_loss
        # loss = model_loss + custom_loss
        # scaler.scale(loss).backward()
        # scaler.step(optimizer)
        # scaler.update()
        # print(loss,model_loss,custom_loss)
        # print(type(loss),type(model_loss),type(custom_loss))

        optimizer.step()
        scheduler.step()
        train_losses.append(loss.detach())
        # break

    # break

    # Logging training loss
    train_losses = [loss.item() for loss in train_losses]
    train_loss = np.mean(train_losses)
    print(f"Train loss: {train_loss} for epoch : {epoch}")
    list_loss_train.append(train_loss)
    # model.save_pretrained(f"{ckpt_dir}/best_ckpt_epoch={epoch}")

    model.eval()
    valid_loss = validation(eval_dataloader, model, batch_size_valid, optimizer, scheduler)
    list_loss_valid.append(valid_loss)

    # Save best model
    if valid_loss < best_loss:
            best_loss = valid_loss
            state_dict = {
                'model_state_dict': model.state_dict(),
                'optim_state_dict': optimizer.state_dict(),
                'sched_state_dict': scheduler.state_dict(),
                'loss': best_loss,
                'epoch': last_epoch
            }

            model.save_pretrained(f"{ckpt_dir}/best_ckpt_epoch={epoch}_valid_loss={round(best_loss, 4)}")

    # break



#################### Epoch: 1 ####################


  0%|          | 0/1092 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 1092/1092 [38:09<00:00,  2.10s/it]


Train loss: 4.309303145923894 for epoch : 1
Validation processing...


100%|██████████| 471/471 [16:33<00:00,  2.11s/it]


Validation loss: 4.237970959608722

#################### Epoch: 2 ####################


...

Validation loss: 4.319615448356434

#################### Epoch: 3 ####################


100%|██████████| 1092/1092 [36:21<00:00,  2.00s/it]


Train loss: 4.261891245623648 for epoch : 3
Validation processing...


100%|██████████| 471/471 [15:58<00:00,  2.04s/it]


Validation loss: 4.228064946814454


In [ ]:
model.save_pretrained("peft_ckpt_epoch(3)")

# Test

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import gdown
import zipfile
import torch

In [11]:
#download the dataset
# url = 'https://drive.google.com/16lncpKpwbxkDM1wFssG5-blx3Yjqtm-q'
# output = 'PEFT_PLASMA'
# gdown.download(url, output, quiet=False)



Downloading...
From: https://drive.google.com/16lncpKpwbxkDM1wFssG5-blx3Yjqtm-q
To: /content/PEFT_PLASMA
1.65kB [00:00, 3.78MB/s]


'PEFT_PLASMA'

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = PeftModel.from_pretrained(base_model, "peft_plasma")
model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (

In [34]:
test_file = "/content/dataset/PUMA/test.json"
TEST_BATCH_SIZE =4
with open(test_file, 'r') as f:
    test_data = json.load(f)

test_dataset = CustomDataset(test_data, tokenizer,max_length = tokenizer.model_max_length)
test_dataloader =  test_create_dataloader(test_dataset = test_dataset,TEST_BATCH_SIZE = TEST_BATCH_SIZE )


In [35]:
gen = []
actual =[]
test_losses = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        output = model(input_ids= input_ids,attention_mask=attention_mask,labels=labels)
        custom_loss,gen_sum = compute_custom_loss(model,input_ids,attention_mask, batch["perspective"])
        loss = output.loss + custom_loss

        gen.extend(gen_sum)
        actual.extend(batch['Summary'])
        test_losses.append(loss.item())

test_loss = np.mean(test_losses) if len(test_losses) > 0 else 0.0
print(f"Validation loss: {test_loss}")


  0%|          | 0/313 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 313/313 [11:15<00:00,  2.16s/it]

Validation loss: 4.127837106823541


In [37]:
df = pd.DataFrame(list(zip(gen,actual)),columns = ['Prediction','Actual'])
df.to_csv('summaryList.csv',index=False)

# Evaluation

In [38]:
class EvaluationMetrics:
    def __init__(self, predictions, references):
        self.predictions = predictions
        self.references = references

    def compute_bleu_scores(self):
        bleu_1, bleu_2, bleu_3, bleu_4 = [], [], [], []
        smoothie = SmoothingFunction().method4
        for prediction, reference in zip(self.predictions, self.references):
            reference = [reference.split()]  # BLEU score API requires tokenized sentences
            prediction = prediction.split()

            bleu_1.append(sentence_bleu(reference, prediction, weights=(1, 0, 0, 0), smoothing_function=smoothie))
            bleu_2.append(sentence_bleu(reference, prediction, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie))
            bleu_3.append(sentence_bleu(reference, prediction, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie))
            bleu_4.append(sentence_bleu(reference, prediction, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie))

        results = {
            "bleu_1": np.mean(bleu_1),
            "bleu_2": np.mean(bleu_2),
            "bleu_3": np.mean(bleu_3),
            "bleu_4": np.mean(bleu_4)
        }

        return results

    def compute_bertscore(self, predictions, references, lang="en"):
        bert_score = evaluate.load('bertscore')
        results =  bert_score.compute(predictions = self.predictions, references = self.references, lang=lang)
        new_res = {}
        new_res['precision'] = np.mean(results['precision'])
        new_res['recall'] = np.mean(results['recall'])
        new_res['f1'] = np.mean(results['f1'])
        return new_res

In [49]:
df2 = pd.read_csv('/content/summaryList(Plasma).csv')
predictions = df2['Prediction'].tolist()
references = df2['Actual'].tolist()
print(predictions[0])
print(references[0])
eval_metrics = EvaluationMetrics(predictions, references)

bleu_scores = eval_metrics.compute_bleu_scores()
bert_scores = eval_metrics.compute_bertscore(predictions, references)

print("BLEU scores:", bleu_scores)
print("Bert score:", bert_scores)

aline of sexual stimulation is an orgasm is climax. Orgasm is a pleasurpriorgasm is asurable physical, also known as aable physical, psychologically known as a , psychological or emotional orgasms sexual o emotional response to climax, climax.
For information purposes, an orgasm is a pleasurable response to sexual stimulation, involving physical, emotional, and sometimes spiritual reactions. Both genders can experience it, but the specific response varies. It's the third stage in the human sexual response cycle, influenced by physical touch, emotional connection, mental intimacy, and complete involvement in the moment. Various factors like thoughts of procreation, power dynamics, and genuine love can contribute to the experience. While having orgasms with someone you love can be great, it doesn't guarantee the long-term success of a relationship.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU scores: {'bleu_1': np.float64(0.03285325786380657), 'bleu_2': np.float64(0.018047390901428727), 'bleu_3': np.float64(0.011636023677512597), 'bleu_4': np.float64(0.007919296872623997)}
Bert score: {'precision': np.float64(0.812429613722399), 'recall': np.float64(0.8097465522944356), 'f1': np.float64(0.8107079293209905)}


In [44]:
for i in bleu_scores:
  print(i,":",bleu_scores[i])
for i in bert_scores:
  print(i,":",bert_scores[i])

bleu_1 : 0.03285325786380657
bleu_2 : 0.018047390901428727
bleu_3 : 0.011636023677512597
bleu_4 : 0.007919296872623997
precision : 0.812429613722399
recall : 0.8097465522944356
f1 : 0.8107079293209905
